In [1]:
import numpy as np
import os
import tempfile
import subprocess
from ultralytics import YOLO
import torch
import onnx2tf
import onnx
import onnx2keras
from onnx2keras import onnx_to_keras
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

Using TensorFlow backend


# Helper Functions


In [4]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size


def convert_bytes(size, unit=None):
    if unit == "KB":
        return print("File size: " + str(round(size / 1024, 3)) + " Kilobytes")
    elif unit == "MB":
        return print("File size: " + str(round(size / (1024 * 1024), 3)) + " Megabytes")
    else:
        return print("File size: " + str(size) + " bytes")


def c_style_hexdump(input, ouput, name):
    with open(input, "rb") as f:
        file = f.read()

    file = bytearray(file)
    _bytes = [f"0x{x:02x}" for x in file]
    file = ",".join(_bytes)

    with open(ouput, "w") as f:
        f.write("#pragma once\n")
        f.write("#include <stdalign.h>\n")
        f.write(f"alignas(16) const unsigned char {name}[] = {{{file}}};")

    return len(_bytes)


def build_header(output, names_with_sizes):
    with open(output, "w") as f:
        f.write('#pragma once\n#ifdef __cplusplus\nextern "C"\n{\n#endif\n')
        f.write("#include <stdalign.h>\n\n")
        for name, size in names_with_sizes:
            f.write(f"alignas(16) extern const unsigned char {name}[{size}];\n")
        f.write("\n#ifdef __cplusplus\n}\n#endif\n")


def evaluate_model(interpreter, x_test, y_test):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for i, test_image in enumerate(x_test):
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print("\n")
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == y_test).mean()
    return accuracy

# Load the Model


In [41]:
model = YOLO("yolov8n.pt")
results = model.val(data="coco128.yaml")

Ultralytics YOLOv8.2.11 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]


                   all        128        929       0.64      0.537      0.605      0.446
                person        128        254      0.797      0.677      0.764      0.538
               bicycle        128          6      0.514      0.333      0.315      0.264
                   car        128         46      0.813      0.217      0.273      0.168
            motorcycle        128          5      0.687      0.887      0.898      0.685
              airplane        128          6       0.82      0.833      0.927      0.675
                   bus        128          7      0.491      0.714      0.728      0.671
                 train        128          3      0.534      0.667      0.706      0.604
                 truck        128         12          1      0.332      0.473      0.297
                  boat        128          6      0.226      0.167      0.316      0.134
         traffic light        128         14      0.734        0.2      0.202      0.139
             stop sig

In [11]:
model.export(format="tflite", imgsz=640, int8=True, data="coco.yaml")

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.9s, saved as 'yolov8n.onnx' (12.3 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=coco128.yaml'


Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

TensorFlow SavedModel: WARNING ⚠️ >300 images recommended for INT8 calibration, found 128 images.
TensorFlow SavedModel: export failure ❌ 0.9s: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3

In [16]:
model.export(format="tflite", imgsz=640, data="coco128.yaml")

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.7s, saved as 'yolov8n.onnx' (12.3 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.17.5...


2024-05-09 23:46:01.784311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 23:46:01.784373: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-05-09 23:46:01.784513: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-05-09 23:46:01.784982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 23:46:01.785039: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

TensorFlow SavedModel: export success ✅ 59.6s, saved as 'yolov8n_saved_model' (30.9 MB)

TensorFlow Lite: starting export with tensorflow 2.13.1...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8n_saved_model/yolov8n_float32.tflite' (12.3 MB)

Export complete (61.8s)
Results saved to /home/eduard/Github/x-heep-femu-tflite-sdk/sw/riscv/apps/tflite_yolo
Predict:         yolo predict task=detect model=yolov8n_saved_model/yolov8n_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=yolov8n_saved_model/yolov8n_float32.tflite imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8n_saved_model/yolov8n_float32.tflite'

## Convert from onnx to tflite

In [2]:
onnx2tf.convert("yolov8n.onnx", output_integer_quantized_tflite=True, output_h5=True)


Model optimizing started ============================================================
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 8              │ 8                │
│ Concat     │ 18             │ 18               │
│ Constant   │ 144            │ 144              │
│ Conv       │ 64             │ 64               │
│ Div        │ 1              │ 1                │
│ MaxPool    │ 3              │ 3                │
│ Mul        │ 58             │ 58               │
│ Reshape    │ 5              │ 5                │
│ Resize     │ 2              │ 2                │
│ Sigmoid    │ 58             │ 58               │
│ Slice      │ 20             │ 20               │
│ Softmax    │ 1              │ 1                │
│ Sub        │ 2              │ 2                │
│ Transpose  │ 1              │ 1                │

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


INT8 Quantization tflite output complete!


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


Full INT8 Quantization tflite output complete!
INT8 Quantization with int16 activations tflite output complete!
Full INT8 Quantization with int16 activations tflite output complete!


## Convert from keras model to tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras.saving.load_model())

## Convert from saved_model to tflite

In [6]:
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 640, 640, 3)
        yield [data.astype(np.float32)]
    
    # # Use coco128.yaml
    # with open("coco128.yaml", "r") as f:
    #     data = yaml.load(f, Loader=yaml.FullLoader)
    # for i in tqdm(range(100)):
    #     img_path = data["val"][i]
    #     img = cv2.imread(img_path)
    #     img = cv2.resize(img, (640, 640))
    #     img = img / 255.0
    #     img = np.expand_dims(img, axis=0)
    #     yield [img.astype(np.float32)]

In [19]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# converter.representative_dataset = representative_dataset

# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8

file = converter.convert()

_, temp = tempfile.mkstemp(".tflite")

with open(temp, "wb") as f:
    f.write(file)

convert_bytes(get_file_size(temp), "MB")

File size: 3.34 Megabytes


In [20]:
tf.lite.experimental.Analyzer.analyze(temp)

=== /tmp/tmpdx_el4w1.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the PAD op takes
tensor #0 and tensor #13 as input and produces tensor #157 as output.

Subgraph#0 main(T#0) -> [T#409]
  Op#0 PAD(T#0, T#13[0, 0, 1, 1, 1, ...]) -> [T#157]
  Op#1 CONV_2D(T#157, T#94, T#18) -> [T#158]
  Op#2 LOGISTIC(T#158) -> [T#159]
  Op#3 MUL(T#158, T#159) -> [T#160]
  Op#4 PAD(T#160, T#13[0, 0, 1, 1, 1, ...]) -> [T#161]
  Op#5 CONV_2D(T#161, T#95, T#19) -> [T#162]
  Op#6 LOGISTIC(T#162) -> [T#163]
  Op#7 MUL(T#162, T#163) -> [T#164]
  Op#8 CONV_2D(T#164, T#96, T#20) -> [T#165]
  Op#9 LOGISTIC(T#165) -> [T#166]
  Op#10 MUL(T#165, T#166) -> [T#167]
  Op#11 STRIDED_SLICE(T#167, T#93[0, 0, 0, 16], T#92[0, 0, 0, 32], T#91[1, 1, 1, 1]) -> [T#168]
  Op#12 CONV_2D(T#168, T#97, T#21) -> [T#169]
  Op#13 LOGISTIC(T#169) -> [T#170]
  Op#14 MUL(T#169, T#170) -> [T#171]
  Op#15 CONV_2D(T#171, T#98, T#22) -> [T#17

In [23]:
# validate on COCO dataset FP32
tflite_yolo = YOLO("saved_model/yolov8n_float32.tflite")
results_converted = tflite_yolo.val(data="coco128.yaml")

# validate on COCO dataset INT8
tflite_yolo = YOLO(temp)
results_quant = tflite_yolo.val(data="coco128.yaml")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics YOLOv8.2.11 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)
Loading saved_model/yolov8n_float32.tflite for TensorFlow Lite inference...
WARNING ⚠️ Metadata not found for 'model=saved_model/yolov8n_float32.tflite'
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:18<00:00,  7.05it/s]


                   all        128        929      0.651      0.532      0.606      0.452
                person        128        254      0.805      0.667      0.764      0.543
               bicycle        128          6      0.661      0.328      0.329      0.232
                   car        128         46      0.817      0.196      0.269      0.181
            motorcycle        128          5      0.602        0.8       0.88      0.672
              airplane        128          6      0.756      0.667      0.845      0.619
                   bus        128          7      0.539      0.714      0.698      0.625
                 train        128          3      0.525      0.667       0.83      0.764
                 truck        128         12      0.689       0.25       0.42      0.239
                  boat        128          6      0.197      0.167      0.331      0.145
         traffic light        128         14      0.684      0.157      0.202      0.139
             stop sig

val: Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:17<00:00,  7.45it/s]


                   all        128        929      0.669      0.536      0.609      0.454
                person        128        254       0.81      0.661      0.757      0.541
               bicycle        128          6      0.655      0.322      0.334      0.239
                   car        128         46       0.77      0.196      0.271      0.184
            motorcycle        128          5       0.63        0.8      0.866      0.709
              airplane        128          6      0.747      0.667      0.823      0.623
                   bus        128          7      0.552      0.714      0.722      0.637
                 train        128          3      0.547      0.667      0.863      0.784
                 truck        128         12      0.783      0.306      0.435      0.258
                  boat        128          6      0.198      0.167      0.342      0.145
         traffic light        128         14          1      0.182       0.22      0.141
             stop sig

## Compare results

In [42]:
from rich.pretty import pprint

print("Original")
pprint(results.results_dict)

pprint("FP32")
pprint(results_converted.results_dict)

pprint("INT8")
pprint(results_quant.results_dict)


Original


{
│   'metrics/precision(B)': 0.6395485765672763,
│   'metrics/recall(B)': 0.5367953634688981,
│   'metrics/mAP50(B)': 0.6048726076857669,
│   'metrics/mAP50-95(B)': 0.44582350082013184,
│   'fitness': 0.46172841150669536
}

'FP32'

{
│   'metrics/precision(B)': 0.6508631511845091,
│   'metrics/recall(B)': 0.5323318534118441,
│   'metrics/mAP50(B)': 0.6059393732573817,
│   'metrics/mAP50-95(B)': 0.4523548364001411,
│   'fitness': 0.46771329008586515
}

'INT8'

{
│   'metrics/precision(B)': 0.6693744628674021,
│   'metrics/recall(B)': 0.5357106358126033,
│   'metrics/mAP50(B)': 0.6089456673483312,
│   'metrics/mAP50-95(B)': 0.4540750785376153,
│   'fitness': 0.4695621374186869
}

## Convert onnx to keras


In [19]:
onnx_model = onnx.load("yolov8n.onnx")
k_model = onnx2keras.onnx_to_keras(onnx_model, ["input_0"])

AttributeError: Current node is not in weights / model inputs / layers.

## Fit a YOLOv8 model with keras directly

https://keras.io/examples/vision/yolov8/


In [36]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")
# https://github.com/keras-team/keras-cv/issues/1886

ValueError: The channel dimension of the inputs should be defined. The input_shape received is (None, None, None, 5), where axis -3 (0-based) is the channel dimension, which found to be `None`.